
# Quick Start - pyCityLayers


# Access to External (Quebec) Data

#### --------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
import numpy as np
import json
from pprint import pprint
import fiona
from shapely.geometry import Polygon, Point, MultiPolygon
from pycitylayers.client import Client
from pycitylayers.utils import PointGQL, PolygonGQL


### Create an instance of client to database

In [ ]:
client = Client().create(source="quebec")


In [ ]:
coll = client.collection
print(coll) 

In [ ]:
sub_coll, indices = coll.search(by_name='crim', return_indices=True)
sub_coll

In [ ]:
ds = sub_coll[0]
ds

In [ ]:
ds.tables

In [ ]:
tb = ds[1]
tb

In [ ]:
tb.columns

In [ ]:
tb.num_rows

### Get N rows from a table given a list of columns 

In [ ]:
query_options = {
    'columns': ['LONGITUDE', 'LATITUDE', 'PDQ'], 
    'nrows': 10000, 
    'skiprows': 0,
}

data = tb.query_simple( **query_options )

data

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

list_of_points = [ Point((float(row['LONGITUDE']), float(row['LATITUDE']))) for _, row in data.iterrows() ]

gdf = gpd.GeoDataFrame(index=list(range(len(list_of_points))), crs='epsg:4326', geometry=list_of_points)

# load neighbourhood shape file
mtl_neighs = gpd.read_file('montreal_neighs.geojson')


fig, ax = plt.subplots(figsize=(12, 10))
sizes = data['PDQ'].apply(lambda x: int(x)*0.1)
gdf.plot(ax=ax, color='b', markersize=sizes, alpha=0.8)
mtl_neighs.plot(ax=ax, color=None, alpha=0.22, facecolor='white', edgecolor='black', linewidth=1)


ax.set_xlim(-74.05, -73.4)
ax.set_ylim(45.35, 45.75)



### Advanced Query


In [ ]:
sql_string = f'SELECT "num_route", "conviviali" FROM "{tb._id}" LIMIT 6 '
print(sql_string)

data = tb.query_sql(sql_string)

data

## save to file

In [ ]:
tb = ds[2]
tb

In [ ]:
tb.to_file(output_dir="downloads")